In [3]:
from dotenv import load_dotenv
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings

load_dotenv()
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    collection_name="constitucion_argentina_v2",
    persist_directory="vector_store",
    embedding_function=embedding
)

In [33]:
from langchain.prompts import ChatPromptTemplate

template = """
Eres un abogado especializado en derecho constitucional argentino y debes utilizar la Constitución Argentina y la jurisprudencia relevante para responder preguntas y resolver dudas relacionadas con derechos y obligaciones legales de los ciudadanos argentinos. Deberá proporcionar información y asesoramiento profesional sobre si los derechos de un individuo están siendo infringidos y si existen fundamentos para iniciar acciones legales en distintas situaciones. El agente debe ofrecer respuestas claras, precisas y fundamentadas, al nivel de un profesional experimentado en el ámbito del derecho.
La respuesta debe estar amparada en alguno de los articulos de la constitucion que se detallan a continuacion:
{articulos_de_la_constitucion}
Reponde la siguiente pregunta con total seguridad y fundamentos en menos de 4 parrafos y sin sugerir consultas a otros profesionales:
{pregunta}
"""

In [34]:
from langchain.schema import StrOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnableMap

prompt = ChatPromptTemplate.from_template(template=template)
retriever = vectordb.as_retriever(search_type="mmr", k=5, fetch_k=10)
model = ChatOpenAI(temperature=0.2)
output_parser = StrOutputParser()

chain = RunnableMap({
    "articulos_de_la_constitucion": lambda x: retriever.get_relevant_documents(x["pregunta"]),
    "pregunta": lambda x: x["pregunta"]
}) | prompt | model | output_parser

In [35]:
respuesta = chain.invoke({
    "pregunta": "¿Puedo salir de mi casa?"
})

In [38]:
print(respuesta)

Según el artículo 14 de la Constitución Argentina, todos los habitantes tienen el derecho de entrar, permanecer, transitar y salir del territorio argentino. Sin embargo, es importante tener en cuenta que este derecho puede ser limitado en situaciones excepcionales, como lo establece el artículo 23 en caso de conmoción interior o ataque exterior que ponga en peligro el ejercicio de la Constitución y las autoridades. En tales casos, se puede declarar el estado de sitio en la provincia o territorio afectado, suspendiendo temporalmente las garantías constitucionales. Durante esta suspensión, el presidente de la República tiene la facultad de arrestar o trasladar a las personas dentro del territorio nacional. Por lo tanto, es necesario evaluar si existe alguna situación excepcional que justifique la limitación del derecho a salir de la casa. En caso de duda, se recomienda consultar con un abogado especializado en derecho constitucional argentino para obtener asesoramiento específico sobre l

In [39]:
respuesta = chain.invoke({
    "pregunta": "¿Puedo cortar la calle para realizar una manifestacion ya que mi empleador no me quiere subir el salario?"
})

In [41]:
print(respuesta)

Según el artículo 14 bis de la Constitución Argentina, todo trabajador tiene derecho a condiciones dignas y equitativas de labor, retribución justa y salario mínimo vital móvil. Sin embargo, el derecho a manifestarse no implica el derecho a cortar la calle. La Constitución garantiza el derecho a la libre expresión y a la protesta pacífica, pero también establece que estas acciones deben realizarse dentro de los límites de la ley y sin afectar los derechos de terceros. 

En este caso, si consideras que tu empleador está incumpliendo con sus obligaciones salariales, lo más recomendable es buscar una solución a través de los mecanismos legales establecidos, como la negociación colectiva, la conciliación o el arbitraje. También puedes recurrir a la justicia laboral para hacer valer tus derechos. Sin embargo, cortar la calle no sería una medida legalmente respaldada y podría generar consecuencias negativas tanto para ti como para los demás ciudadanos. Es importante buscar alternativas legal

In [42]:
respuesta = chain.invoke({
    "pregunta": "¿A que hace referencia el articulo 18 sobre el principio de legalidad?"
})

In [43]:
print(respuesta)

El artículo 18 de la Constitución Argentina hace referencia al principio de legalidad en el ámbito penal. Establece que ningún habitante de la Nación puede ser penado sin juicio previo fundado en ley anterior al hecho del proceso. Esto significa que nadie puede ser condenado por un delito sin que exista una ley que lo establezca como tal antes de que se haya cometido el acto delictivo.

Además, el artículo 18 prohíbe que los ciudadanos sean juzgados por comisiones especiales o sacados de los jueces designados por la ley antes del hecho de la causa. Esto garantiza que los ciudadanos tengan derecho a un juicio justo y a ser juzgados por jueces imparciales y competentes.

El principio de legalidad también implica que nadie puede ser obligado a declarar contra sí mismo y que nadie puede ser arrestado sino en virtud de una orden escrita de autoridad competente. Asimismo, el artículo 18 establece la inviolabilidad de la defensa en juicio de la persona y de los derechos, así como la inviolabi

In [44]:
respuesta = chain.invoke({
    "pregunta": "¿Que pasa con los tratados internacionales firmados por Argentina, tiene algun rango de jerarquía frente a la constitucion?"
})

In [45]:
print(respuesta)

Según el artículo 31 de la Constitución Argentina, los tratados internacionales firmados por Argentina tienen rango de ley suprema de la Nación, al igual que la propia Constitución y las leyes dictadas por el Congreso. Esto implica que los tratados internacionales tienen jerarquía constitucional y deben ser respetados y cumplidos por todas las autoridades y ciudadanos argentinos. Por lo tanto, en caso de existir un conflicto entre un tratado internacional y una ley o disposición constitucional, prevalecerá el tratado internacional. Esto significa que los derechos y obligaciones establecidos en los tratados internacionales deben ser garantizados y protegidos por el Estado argentino, y cualquier violación a estos derechos podría ser motivo de acciones legales.


In [46]:
respuesta = chain.invoke({
    "pregunta": "¿A que hace referecia el articulo 14 bis sobre el derecho de los trabajadores?"
})